In [1]:
import networkx as nx
from gerrychain import Graph
import math
import time
import gurobipy as gp
from gurobipy import GRB

from util import update_attributes, get_k_L_U
from cluster import max_cluster_main
filepath = 'C:\\districting-data-2020\\'

In [2]:
from number_of_districts import congressional_districts_2020
states = sorted([ state for state in congressional_districts_2020.keys() ])

results = dict()

for district_type in ['SH','SS', 'CD']:
    
    print("results =",results)
    
    print("**********************************")
    print("**********************************")
    print("District_type:",district_type)
    print("**********************************")
    print("**********************************")
    print("")
    
    for state in ['WY']:   
        
        print("**********************************")
        print("State:",state)
        print("**********************************")

        filename = state + '_county.json'
        GC = Graph.from_json( filepath + filename )
        update_attributes(GC, state)    

        (k, L, U) = get_k_L_U(GC, state, district_type)
        if k <= 1 or not nx.is_connected(GC):
            print("Skipping this state because k <= 1 or because G is disconnected.")
            continue

        r = 3 if state == 'TX' or district_type == 'CD' else 0
        results[state,district_type] =  max_cluster_main(GC, L, U, k, restarts=r, time_limit=24*3600)


results = {}
**********************************
**********************************
District_type: SH
**********************************
**********************************

**********************************
State: WY
**********************************
Starting WY with k = 62 and deviation = 0.1
Thus, we have L = 8839 and U = 9769
Initially, cluster_UB = 13

********************************************************
After local search, # clusters, #cut edges = 1 0
********************************************************

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-27
Set parameter TimeLimit to value 86400
Set parameter IntFeasTol to value 1e-07
Set parameter FeasibilityTol to value 1e-07
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 2051 rows, 2898 

In [3]:
print("state type cluster_LB cluster_UB gap")
for (state, district_type) in results.keys():
    clusters = results[state, district_type]['clusters']
    sizes = results[state, district_type]['sizes']
    cluster_UB = results[state, district_type]['cluster_UB']
    print(state, district_type, len(clusters), cluster_UB, cluster_UB-len(clusters))

state type cluster_LB cluster_UB gap
WY SH 10 10 0
WY SS 9 9 0


In [4]:
print("results =",results)

results = {('WY', 'SH'): {'initial_UB': 13, 'heuristic_time': '0.00', 'heuristic_num_clusters': 1, 'heuristic_num_cut_edges': 0, 'heuristic_iterations': 0, 'MIP_time': '0.75', 'cleanup_time': '19.19', 'clusters': [[1], [0], [5, 9, 13, 16], [6, 7, 15, 17], [4], [10, 12], [2, 3, 8, 11, 20], [14, 19], [18, 21], [22]], 'sizes': [11, 9, 10, 6, 4, 3, 4, 7, 3, 5], 'num_clusters': 10, 'num_cut_edges': 33, 'cluster_UB': 10}, ('WY', 'SS'): {'initial_UB': 10, 'heuristic_time': '0.00', 'heuristic_num_clusters': 1, 'heuristic_num_cut_edges': 0, 'heuristic_iterations': 0, 'MIP_time': '1.19', 'cleanup_time': '12.70', 'clusters': [[0, 18], [1, 11], [2, 3, 8], [4], [19, 21], [9, 10, 12, 14], [5, 13, 16], [6, 7, 15, 17], [20, 22]], 'sizes': [5, 6, 1, 2, 3, 4, 4, 3, 3], 'num_clusters': 9, 'num_cut_edges': 33, 'cluster_UB': 9}}


In [5]:
print("state type cluster_LB cluster_UB gap")
for (state, district_type) in results.keys():
    clusters = results[state, district_type]['clusters']
    sizes = results[state, district_type]['sizes']
    cluster_UB = results[state, district_type]['cluster_UB']
    if len(sizes) < cluster_UB:
        print(state, district_type, len(clusters), cluster_UB, cluster_UB-len(clusters))

state type cluster_LB cluster_UB gap
